In [6]:

data_root = "/home/maling/fanqiliang/data/Prostate-MRI-US-Biopsy"
stl_root = "/home/maling/fanqiliang/data/Prostate-MRI-US-Biopsy/STLs/STLs"

us_root = "/home/maling/fanqiliang/output_data/medical_data/us"
mr_root = "/home/maling/fanqiliang/output_data/medical_data/mr"


an_root = "/home/maling/fanqiliang/data/Prostate-MRI-US-Biopsy/manifest-1599764098812"
annoation = "/home/maling/fanqiliang/data/Prostate-MRI-US-Biopsy/manifest-1599764098812/metadata.csv"
import pandas as pd
dt = pd.read_csv(annoation)
print(dt.columns)
# STL使用series UID来对应
print(dt.head(1)["Series UID"].values)
print(dt.head(1)["Study UID"].values)
dt.head(2)

Index(['Series UID', 'Collection', '3rd Party Analysis',
       'Data Description URI', 'Subject ID', 'Study UID', 'Study Description',
       'Study Date', 'Series Description', 'Manufacturer', 'Modality',
       'SOP Class Name', 'SOP Class UID', 'Number of Images', 'File Size',
       'File Location', 'Download Timestamp'],
      dtype='object')
['1.3.6.1.4.1.14519.5.2.1.266717969984343981963002258381778490221']
['1.3.6.1.4.1.14519.5.2.1.85548304921965658367726869399297351743']


,Series UID,Collection,3rd Party Analysis,Data Description URI,Subject ID,Study UID,Study Description,Study Date,Series Description,Manufacturer,Modality,SOP Class Name,SOP Class UID,Number of Images,File Size,File Location,Download Timestamp
0,1.3.6.1.4.1.14519.5.2.1.2667179699843439819630...,Prostate-MRI-US-Biopsy,NaN,https://doi.org/10.7937/TCIA.2020.A61IOC1A,Prostate-MRI-US-Biopsy-0001,1.3.6.1.4.1.14519.5.2.1.8554830492196565836772...,MRI PROSTATE W WO CONTRAST,06-28-2009,t2spcrstaxial oblProstate,SIEMENS,MR,MR Image Storage,1.2.840.10008.5.1.4.1.1.4,60,7.70 MB,.\Prostate-MRI-US-Biopsy\Prostate-MRI-US-Biops...,2021-06-12T16:52:28.274
1,1.3.6.1.4.1.14519.5.2.1.1202285930413120999892...,Prostate-MRI-US-Biopsy,NaN,https://doi.org/10.7937/TCIA.2020.A61IOC1A,Prostate-MRI-US-Biopsy-0001,1.3.6.1.4.1.14519.5.2.1.3019932889266692284498...,3D Rendering,11-28-2010,NaN,Eigen,US,Ultrasound Multi-frame Image Storage,1.2.840.10008.5.1.4.1.1.3.1,1,28.06 MB,.\Prostate-MRI-US-Biopsy\Prostate-MRI-US-Biops...,2021-06-12T16:52:45.464


In [8]:
import os
from glob import glob
import SimpleITK as sitk
import shutil

if not os.path.exists(us_root):
    os.makedirs(us_root)
if not os.path.exists(mr_root):
    os.makedirs(mr_root)

# US (3D -> 2D) 

In [17]:
us_dt = dt[dt["Modality"] == "US"]
print("the number of us files", len(us_dt))
files = us_dt["File Location"].values
sids = us_dt["Series UID"].values
assert len(files) == len(sids)

from typing import cast
for idx, (f, sid) in enumerate(zip(files, sids)):
    f = cast(str, f)
    f = f.lstrip(".\\").replace("\\", "/")
    f = os.path.join(an_root, f)

    save_path = os.path.join(us_root, f"{idx}")
    data_path = os.path.join(save_path, "data")
    if not os.path.exists(data_path):
        os.makedirs(data_path, exist_ok=True)
    dicom = glob(os.path.join(f, "*.dcm"))[0]
    img = sitk.ReadImage(dicom)
    arr = sitk.GetArrayFromImage(img)
    for arr_idx, slice in enumerate(arr):
        save_file = os.path.join(data_path, f"{arr_idx}.dcm")
        sitk.WriteImage(sitk.GetImageFromArray(slice), save_file)

    stls = glob(os.path.join(stl_root, f"*{sid}.STL"))
    for stl in stls:
        shutil.copy(stl, stl.replace(stl_root, save_path))


the number of us files 1762


# MR (2D already)

In [24]:
mr_dt = dt[dt["Modality"] == "MR"]
print("the number of MR files", len(mr_dt))
files = mr_dt["File Location"].values
sids = mr_dt["Series UID"].values
assert len(files) == len(sids)

from typing import cast
for idx, (f, sid) in enumerate(zip(files, sids)):
    f = cast(str, f)
    f = f.lstrip(".\\").replace("\\", "/")
    f = os.path.join(an_root, f)

    save_path = os.path.join(mr_root, f"{idx}")
    data_path = os.path.join(save_path, "data")
    if not os.path.exists(data_path):
        os.makedirs(data_path, exist_ok=True)
    dicoms = glob(os.path.join(f, "*.dcm"))
    for idx, d in enumerate(dicoms):
        save_file = os.path.join(data_path, os.path.basename(d))
        shutil.copy(d, save_file)

    stls = glob(os.path.join(stl_root, f"*{sid}.STL"))
    for stl in stls:
        shutil.copy(stl, stl.replace(stl_root, save_path))


the number of MR files 1017


# Stat
MR: 59293

US: 435628

CT: 159380